<a href="https://colab.research.google.com/github/WWWonderer/Dataset_studies/blob/main/QA/FM2/FM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Raw jsonl



In [23]:
%%capture
!wget https://raw.githubusercontent.com/google-research/fool-me-twice/main/dataset/train.jsonl -O train.jsonl
!wget https://raw.githubusercontent.com/google-research/fool-me-twice/main/dataset/dev.jsonl -O dev.jsonl

In [38]:
import json
with open('dev.jsonl', 'rb') as f:
  for i, line in enumerate(f, start=1):
    try:
      json_obj = json.loads(line.decode('utf-8'))
      print(json.dumps(json_obj, indent=2))
      break
    except json.JSONDecodeError:
      pass

{
  "category": "Popular Culture",
  "correct_votes": 1,
  "gold_evidence": [
    {
      "section_header": "Production",
      "text": "Then prime minister Indira Gandhi declared a state of emergency in India and shooting would be impossible."
    }
  ],
  "id": "01EICaMMy6uOPHdoEGAf",
  "label": "SUPPORTS",
  "retrieved_evidence": [
    {
      "section_header": "Release and reception | Critical response",
      "text": "At a time of deep political unrest, economic dislocation and nuclear anxiety, seeing \"Gandhi\" is an experience that will change many minds and hearts."
    },
    {
      "section_header": "Release and reception | Box office",
      "text": "Due to the running time, it could only be shown three times a day."
    },
    {
      "section_header": "Release and reception | Critical response",
      "text": ", Jack Kroll stated that \"There are very few movies that absolutely must be seen."
    },
    {
      "section_header": "Release and reception",
      "text": "Gan

In [25]:
#@title Load JSONL Data (official method)
import requests

LABELS = {'SUPPORTS': '0', 'REFUTES': '1'}

# in official code, this is an inner function to generate a list of data tuples
# the data tuples will then be converted to tensorflow Dataset (not shown here)
def _load_json(url_path):
  data_tuples = []

  data = requests.get(url_path)

  for line in data.content.decode('utf-8').split('\n'):
    if not line:
      continue
    json_line = json.loads(line)
    # To work with CloudTPU, we cannot use a generator but have to create the
    # data using `from_tensor_slices`.  Because this requires a homogenous
    # tuples, the label is a string at this point, and we only parse it to
    # int later.
    data_tuples.append(
        (json_line['text'],  ' '.join(
            x['text'] for x in json_line['gold_evidence']),
            LABELS[json_line['label']]))

  return data_tuples

data_tuples_train = _load_json('https://raw.githubusercontent.com/google-research/fool-me-twice/main/dataset/train.jsonl')
data_tuples_dev = _load_json('https://raw.githubusercontent.com/google-research/fool-me-twice/main/dataset/dev.jsonl')

In [32]:
print("total tuples dev: ", len(data_tuples_dev))
for i in range(5):
  print("text: ", data_tuples_dev[i][0])
  print("gold evidence: ", data_tuples_dev[i][1])
  print("label: ", data_tuples_dev[i][2])
  print("*" * 1000)

total tuples dev:  1169
text:  Filming for the movie Gandhi in India was delayed due to political unrest.
gold evidence:  Then prime minister Indira Gandhi declared a state of emergency in India and shooting would be impossible.
label:  0
*****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [34]:
data_tuples_dev_support = [data_tuple for data_tuple in data_tuples_dev if data_tuple[2] == '0']
data_tuples_dev_refute = [data_tuple for data_tuple in data_tuples_dev if data_tuple[2] == '1']

In [36]:
print("total tuples dev support: ", len(data_tuples_dev_support))
for i in range(5):
  print("text: ", data_tuples_dev_support[i][0])
  print("gold evidence: ", data_tuples_dev_support[i][1])
  print("label: ", data_tuples_dev_support[i][2])
  print("*" * 1000)

total tuples dev support:  596
text:  Filming for the movie Gandhi in India was delayed due to political unrest.
gold evidence:  Then prime minister Indira Gandhi declared a state of emergency in India and shooting would be impossible.
label:  0
**********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [37]:
print("total tuples dev refute: ", len(data_tuples_dev_refute))
for i in range(5):
  print("text: ", data_tuples_dev_refute[i][0])
  print("gold evidence: ", data_tuples_dev_refute[i][1])
  print("label: ", data_tuples_dev_refute[i][2])
  print("*" * 1000)

total tuples dev refute:  573
text:  Willie Stargell lived with his father after his mother and father  separated.
gold evidence:  Stargell was born in Earlsboro, Oklahoma, but later moved to Florida with an aunt after his parents divorced.
label:  1
*****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************